The function ```dissolve_geometries()``` takes four parameters: ```main_file```, ```join_file```, ```output_file```, and ```log_file```:
- ```main_file``` represents the main GeoJSON file containing the primary geometries.
- ```join_file``` represents the GeoJSON file containing the geometries to join with the main file.
- ```output_file``` is the desired output location for the dissolved data.
- ```log_file``` is the file where log details are recorded.

The function performs the following steps:
1. Loads the main and join GeoJSON files into separate GeoDataFrames.
2. Reprojects the main data to match the CRS of the join data.
3. Performs a spatial join to associate each join feature with the corresponding main feature.
4. Dissolves the join geometries within each main feature into a single polygon per main feature.
5. Saves the dissolved data to a new GeoJSON file.
6. Logs script run details, including the current timestamp, file names, and output file location, to the log file.

The execution times for each step are measured and printed during the execution of the function.

This generalized approach allows you to dissolve geometries from any pair of GeoJSON files based on their spatial relationship and save the result to a new file, while also logging the details of the script run.

In [ ]:
# User input variables
main_file = 'lad_simple.geojson'
join_file = 'msoa_simple.geojson'
output_file = 'dissolved.geojson'
log_file = 'log.csv'


In [ ]:
# Import libraries
import geopandas as gpd
import pandas as pd
import csv
import datetime
import time

In [ ]:
# Define function to dissolve geometries
def dissolve_geometries(main_file, join_file, output_file, log_file):
    total_start_time = time.time()

    start_time = time.time()
    # Load the main and join GeoJSON files into separate GeoDataFrames
    main_data = gpd.read_file(main_file)
    join_data = gpd.read_file(join_file)
    load_time = time.time() - start_time
    print("Loading data: {:.2f} seconds".format(load_time))

    start_time = time.time()
    # Reproject the join data to match the CRS of the main data
    join_data = join_data.to_crs(main_data.crs)
    reprojection_time = time.time() - start_time
    print("Reprojection: {:.2f} seconds".format(reprojection_time))

    start_time = time.time()
    # Perform a spatial join to associate each join feature with the corresponding main feature
    joined_data = gpd.sjoin(join_data, main_data, how='inner', predicate='intersects')
    spatial_join_time = time.time() - start_time
    print("Spatial join: {:.2f} seconds".format(spatial_join_time))

    # Check if the joined DataFrame is empty
    if not joined_data.empty:
        # Extract the attribute names from the main_data GeoDataFrame
        attribute_names = main_data.columns.tolist()

        # Prompt the user for the join_attribute
        print("Available attribute names for join_attribute:", attribute_names)
        time.sleep(0.5)  # Delay to ensure the prompt is displayed
        join_attribute = input("Enter the join attribute name from the available attribute names: ")


        start_time = time.time()
        # Dissolve the join geometries within each main feature into a single polygon per main feature
        dissolved_data = joined_data.dissolve(by=join_attribute)
        dissolve_time = time.time() - start_time
        print("Dissolve: {:.2f} seconds".format(dissolve_time))


        # Identify features that were left out
        left_out = main_data[~main_data.index.isin(dissolved_data.index)]
        # Union the left out features back into the dissolved polygons
        final_data = gpd.GeoDataFrame(pd.concat([dissolved_data, left_out]))
        print("Union undissolved: {:.2f} seconds".format(dissolve_time))


        start_time = time.time()
        # Save the final dissolved data to a new GeoJSON file
        final_data.to_file(output_file, driver='GeoJSON')
        save_time = time.time() - start_time
        print("Saving data: {:.2f} seconds".format(save_time))


        start_time = time.time()
        # Log script run details to the log file
        log_details = [
            datetime.datetime.now(), 
            main_file, 
            join_file, 
            join_attribute, 
            output_file, 
            load_time,
            reprojection_time,
            spatial_join_time,
            dissolve_time,
            save_time,
            time.time() - total_start_time  # Total script execution time
        ]
        write_log(log_file, log_details)
        log_time = time.time() - start_time
        print("Logging: {:.2f} seconds".format(log_time))
    else:
        # Handle the case when the joined DataFrame is empty
        print("No intersections found between the geometries.")


In [ ]:
# Define function to write log details to a CSV file
def write_log(log_file, log_details):
    with open(log_file, 'a', newline='') as file:
        writer = csv.writer(file)

        # Check if the log file is empty
        if file.tell() == 0:
            # Write the header row
            writer.writerow([
                "Timestamp", 
                "Main File", 
                "Join File", 
                "Join Attribute", 
                "Output File", 
                "Load Time",
                "Reprojection Time",
                "Spatial Join Time",
                "Dissolve Time",
                "Save Time",
                "Total Time"
            ])

        # Write the log details
        writer.writerow(log_details)

In [ ]:
dissolve_geometries(main_file, join_file, output_file, log_file)